In [1]:
import pandas as pd
import pickle5 as pickle55

import os

from query_phenomizer import utils

In [2]:
class Update_HPO:
    
    def __init__(self, obo_file):
        self.obo_file = obo_file
        self.replacement_dict = self.create_dictionary_replacements()
        self.non_phenotype_nodes = self.find_non_phenotype_nodes()
   
    def find_non_phenotype_nodes(self):
        non_phenotype_nodes = set(['HP:0000005', 'HP:0012823', 'HP:0040279'])
        
        nodes_added = len(non_phenotype_nodes)
        while nodes_added > 0:

            hpobo = open(self.obo_file)
            nodes_added = 0
            term = ''

            for line in hpobo:
                if line.startswith('id'):
                    term = line.split(': ')[1].split('\n')[0]

                elif line.startswith('is_a'):
                    hpo_term = line.split(': ')[1].split(' !')[0]
                    if hpo_term in non_phenotype_nodes and term not in non_phenotype_nodes:
                        non_phenotype_nodes.add(term)
                        nodes_added += 1
        return non_phenotype_nodes


    def create_dictionary_replacements(self):
        hpobo = open(self.obo_file)
        replacements = {} #key is replaced by value

        term = ''

        for line in hpobo:
            if line.startswith('id'):
                term = line.split(': ')[1].split('\n')[0]

            elif line.startswith('replaced_by'):
                hpo_term = line.split(': ')[1].split('\n')[0]
                replacements[term] = hpo_term

            elif line.startswith('alt_id'):
                hpo_term = line.split(': ')[1].split('\n')[0]
                replacements[hpo_term] = term
        return replacements

    
    def create_dictionary_id_name(self): #gebruik ik helemaal niet
        hpobo = open(self.obo_file)
        id_to_name = {}

        term_id = ''

        for line in hpobo:
            if line.startswith('id'):
                term_id = line.split(': ')[1].split('\n')[0]

            elif line.startswith('name'):
                term_name = line.split(': ')[1].split('\n')[0]
                id_to_name[term_id] = term_name

        return id_to_name
    
    def replace(self, term):
        if term in self.replacement_dict.keys():
            return self.replacement_dict[term]
        else:
            return term

    
    def delete_non_phenotype_nodes(self, term_list):
        new_term_list = [i for i in term_list if i not in self.non_phenotype_nodes]
        return new_term_list


    def update_phenotype(self, patient):
        replaced = [self.replace(term) for term in patient]
        replaced_deleted = self.delete_non_phenotype_nodes(replaced)
        return replaced_deleted

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

file = './data_all.pickle'  
with open(file, "rb") as fh:
      data = pickle55.load(fh)

In [4]:
phenopy_data_directory = os.path.join(os.getenv('HOME'), '.phenopy/data')
obo_file = os.path.join(phenopy_data_directory, 'hp.obo')
hpoa_file = os.path.join(phenopy_data_directory, 'phenotype.hpoa')

In [5]:
updater = Update_HPO(obo_file)
for index in range(len(data['hpo_all'])):
    data['hpo_all'][index] = updater.update_phenotype(data['hpo_all'][index])
    data['hpo_all_with_parents'][index] = updater.update_phenotype(data['hpo_all_with_parents'][index])
    data['label'][index] = data['label'][index].replace("_", "")

In [6]:
omim_genes = {
    'OMIM:148050' : 'ankrd',
    'OMIM:300958' : 'ddx3x',
    'OMIM:610443' : 'kansl1',
    'OMIM:611867' : '22q11',
    'OMIM:614104' : 'dyrk1a',
    'OMIM:615009' : 'pacs1',
    'OMIM:615873' : 'adnp',
    'OMIM:616158' : 'pura',
    'OMIM:616708' : 'wac',
    'OMIM:617140' : 'son',
    'OMIM:618846' : 'kdm3b', 
    'OMIM:618829' : 'spop_2',
    'OMIM:618828' : 'spop_1',
    'OMIM:617854' : 'cltc',
    'OMIM:617557' : 'yy1',
    'OMIM:617450' : 'ppm1d'
}

diseases = list(omim_genes.keys())

## HPOA

In [14]:
hpoa = open(hpoa_file)
omims = []
ids = []

for line in hpoa:
    if line.startswith('OMIM') or line.startswith('ORPHA') or line.startswith("DECIPHER"):
        disease_id = line.split('\t')[0]
        ids.append(disease_id)
    if line.startswith('OMIM'):
        omims.append(disease_id)
    
print("Amount of OMIM disease ID's found in HPOA file:",len(set(omims)))
print("Amount of (OMIM/ORPHA/DICEPHER) disease ID's found in HPO file:", len(set(ids)))

Amount of OMIM disease ID's found in HPOA file: 8120
Amount of (OMIM/ORPHA/DICEPHER) disease ID's found in HPO file: 12367


## Genes

In [8]:
top_n = 100
username = 'fill_in_yourself'
password = 'fill_in_yourself'

In [9]:
gene_predictions = {}
for index in range(len(data['hpo_all'])):
    separator = ', '
    a = (separator.join(data['hpo_all'][index])) 

    result = utils.query_phenomizer(username, password,  a)
    genes = []
    for i in range(top_n):
        genes.append(result.text.split('\n')[6:][i].split('\t')[4])
    
    seperate_genes = [j.split(' ')[0] for j in genes]
    without_empty = list(filter(None, seperate_genes))
    final_genes = [gene.lower() for gene in without_empty]
    gene_predictions[index] = final_genes

## OMIM

In [11]:
diagnoses_database = {}
for index in range(len(data['hpo_all'])):
    separator = ', '
    a = (separator.join(data['hpo_all'][index])) 

    result = utils.query_phenomizer('scout', 'scout123',  a)

    diagnoses_omim = []
    for i in range(top_n):
        omim = result.text.split('\n')[6:][i].split('\t')[2]
        diagnoses_omim.append(omim)

    diagnoses_database[index] = diagnoses_omim

# Correct classification

In [30]:
correct_label_in_top_n = 0

for key in gene_predictions:
    genes = set(gene_predictions[key])
    diagnoses = diagnoses_database[key]
    
    if data['label'][key] in gene_predictions[key]:
        correct_label_in_top_n += 1
        
    else: 
        for j in diagnoses:
            if j in diseases:
                gene = omim_genes[j]
                if gene == data['label'][key]:
                    correct_label_in_top_n += 1

print('Correct labels found in top 100:', correct_label_in_top_n)
print('Accuracy:', 23/553 )

Correct labels found in top 100: 23
Accuracy: 0.04159132007233273
